In [1]:
# Train a CNN regression to predict the mCG level of all DMRs
import pandas as pd
import numpy as np
import time, re
import matplotlib.pyplot as plt
from IPython import display
import timeit

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [2]:
%load_ext line_profiler

In [5]:
use_kmers = True
N1 = 2
N2 = 2

lr = 0.05

In [4]:
fn_save_prefix = 'RegressData/Regress_Feb16_data_'
save_vars = ['genes2enhu', 'rnau', 'df_mlevelu', 'df_atacu', 'genes']
for var in save_vars:
    fn_save = fn_save_prefix+var+'.pkl'
    cmd = '%s=pd.read_pickle("%s")' % (var, fn_save)
    exec(cmd)
    print('Loaded %s' % fn_save)

if use_kmers:
    with np.load('RegressData/Regress_Feb16_data_kmer_countsu.npz', allow_pickle=True) as x:
        kmer_countsu=x['kmer_countsu']

Loaded RegressData/Regress_Feb16_data_genes2enhu.pkl
Loaded RegressData/Regress_Feb16_data_rnau.pkl
Loaded RegressData/Regress_Feb16_data_df_mlevelu.pkl
Loaded RegressData/Regress_Feb16_data_df_atacu.pkl
Loaded RegressData/Regress_Feb16_data_genes.pkl


In [247]:
# Impute the NaN values in mlevel
df_mlevelu_mean = np.outer(df_mlevelu.median(axis=1).values, df_mlevelu.median(axis=0).values)
df_mlevelu_mean = pd.DataFrame(df_mlevelu_mean, index=df_mlevelu.index, columns=df_mlevelu.columns)
df_mlevelu_mean = df_mlevelu_mean.fillna(df_mlevelu.median())
df_mlevelu = df_mlevelu.fillna(df_mlevelu_mean)
assert(not df_mlevelu.isna().any(axis=None))
assert(not df_atacu.isna().any(axis=None))
ensids = rnau.index.values

/cndd/emukamel/conda_envs/scanpy/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1116: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input)


In [248]:
# Filter out bad enhancers
bad_enh = df_mlevelu.loc[genes2enhu.loc[:,'enh_pos']].isna().any(axis=1)
bad_enh = bad_enh | df_atacu.loc[genes2enhu.loc[:,'enh_pos']].isna().any(axis=1)
good_enh = [not i for i in bad_enh]
genes2enhu = genes2enhu[good_enh]
if use_kmers:
    kmer_countsu = kmer_countsu[good_enh]
    K = kmer_countsu.shape[1] # Number of K-mers
else:
    K = 0

In [251]:
# Gather all the mC and ATAC data into a numpy array for quick indexing
# %%timeit
max_nenh = genes2enhu.groupby('ensid')['gene_chr'].count().max()
Ne = ensids.shape[0]
Nc = df_mlevelu.shape[1]
rna_lookup = rnau.loc[ensids].to_numpy()
ml_lookup = np.zeros((Ne,max_nenh,Nc))
atac_lookup = np.zeros((Ne,max_nenh,Nc))
for i,ens in enumerate(ensids):
    curr_enh = genes2enhu.loc[ens,['enh_pos','enh_num']]
    mlu = df_mlevelu.loc[curr_enh['enh_pos']].to_numpy()
    atacu = df_atacu.loc[curr_enh['enh_pos']]
    ml_lookup[i,:mlu.shape[0],:] = mlu
    atac_lookup[i,:atacu.shape[0],:] = atacu
# ensids.shape

In [252]:
rna_lookup = rnau.loc[ensids].to_numpy()
rna_lookup.shape

(3947, 28)

In [253]:
np.savez_compressed('RegressData/Regress_Feb16_data_AllLookups.npz',
                   ml_lookup=ml_lookup,
                   atac_lookup=atac_lookup,
                   rna_lookup=rna_lookup)

In [ ]:
def get_data(ensid):
    # For a list of ensids, return the x and y features
    x = []
    for i,ens in enumerate(ensid):
        curr_enh = genes2enhu.loc[ens,['enh_pos','enh_num']]
#         curr_enh = curr_enh[:1]
#         E = curr_enh.groupby('ensid').count().max(axis=0);
        mlu = df_mlevelu.loc[curr_enh['enh_pos']]
        atacu = df_atacu.loc[curr_enh['enh_pos']]
        Ne,Nc = mlu.shape
        if use_kmers:
            kmeru = kmer_countsu[curr_enh['enh_num']]
            Ne,Nk = kmeru.shape
            kmeru = kmeru[:,np.newaxis,:]
            kmeru = np.broadcast_to(kmeru,(Ne,Nc,Nk))
            x1 = np.dstack((mlu,atacu,kmeru))
        else:
            x1 = np.dstack((mlu,atacu))
        x1 = x1[np.newaxis,:,:,:]
        x.append(x1)
    
    # Pad the dimensions
    max_nenh = np.max([xi.shape[1] for xi in x])
    xpad = []
    for xi in x:
        xpad.append(np.pad(xi, ((0,0),(0,max_nenh-xi.shape[1]),(0,0),(0,0))))
    x = np.concatenate(xpad, axis=0)

    y = rnau.loc[ensid].to_numpy().reshape((-1,Nc))
    
    # Testing:
    for c in range(Nc):
        y[:,c] = y[:,c]*0+c+0.123
    
    x = torch.tensor(x, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.float)
    
    return {'x': x, 'y': y}


def get_data_index(ensid_index):
    # For a list of ensids, return the x and y features
    mlu = ml_lookup[ensid_index,:,:] # Ngenes x Ne x Nc
    atacu = ml_lookup[ensid_index,:,:]
    x = np.stack((mlu,atacu),axis=3)
    y = rna_lookup[ensid_index,:]
    
#     # Testing:
#     for c in range(Nc):
#         y[:,c] = y[:,c]*0+c+0.123
    
    x = torch.tensor(x, dtype=torch.float)
    y = torch.tensor(y, dtype=torch.float)
    
    return {'x': x, 'y': y}

In [ ]:
Nc = df_mlevelu.shape[1]

# Define a class for the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(2+K, N1);
        self.fc2 = nn.Linear(3*N1, N2);
        self.fc3 = nn.Linear(N2*Nc, Nc);
        
    def forward(self, x): 
        x = F.relu(self.fc1(x)) # In: N x Eg x C x (2+K), Out: N x Eg x C x N1
        xmax = torch.max(x, 1)[0]       # Out: N x C x N1
        xmean = torch.mean(x, 1)       # Out: N x C x N1
        xsum = torch.sum(x, 1)       # Out: N x C x N1
        x = torch.cat((xmax,xmean,xsum),2) # Out: N x C x 3*N1
        x = F.relu(self.fc2(x))       # Out: N x C x N2
        x = torch.reshape(x,(-1,N2*Nc)) # Out: N x C
        x = self.fc3(x)
        
        return x

In [ ]:
net = Net()
if device is not "cpu":
    net.to(torch.device(device))
device

In [ ]:
net(get_data(ensids[:3])['x'].to(device)).shape
# get_data(ensids[:15])['x'].shape, get_data(ensids[:2])['y'].shape

In [ ]:
mseloss=nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1)

running_time = 0
loss_vec = corr_vec = []
nepochs = 0

loss_test=np.array([])
loss_train = np.array([])
nparams = np.array([])
l1params = np.array([])

# Train/Test split
test = [c in ['chr10','chr12','chr14'] for c in rnau.join(genes)['chr']];
train = [not i for i in test]

test = np.random.permutation(np.nonzero(test)[0]).squeeze()
train = np.random.permutation(np.nonzero(train)[0]).squeeze()

ensids = rnau.index.values

In [ ]:
import datetime
today=datetime.datetime.today().strftime('%d-%m-%Y')
fn_save = 'Regress_pytorch_N_%d_%d.%s.torch' % (N1,N2,today)
fn_save

In [ ]:
batch_size = 10
i = 10
def testfn():
    indices = train[i:i+batch_size]
    print(indices)

    # Input should be of size: (batch, channels, samples)
#     batch_data = get_data(ensids[indices])
    batch_data = get_data_index(indices)
    batch_X = batch_data['x']
    batch_y = batch_data['y']
#         print(batch_X.shape, batch_y.shape)

    # Send training data to CUDA
    if device is not "cpu":
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(batch_X)
    loss = criterion(outputs, batch_y)
    loss.backward()
    optimizer.step()

In [ ]:
batch_data = get_data_index(indices)
batch_X = batch_data['x'].to(device)
batch_y = batch_data['y']
net(batch_X)

In [ ]:
%lprun -f testfn testfn()

In [ ]:
criterion = nn.MSELoss()
num_epochs = 1000;
batch_size = 64;

loss_train, loss_test = np.array([]), np.array([])
t0 = time.time()
for epoch in range(num_epochs):  # loop over the dataset multiple times
    nsamp = 0
    running_loss = 0.0
    running_time = 0.0
    net.train()
    t0train = time.time()
    for i in range(0, len(train), batch_size):
        tstart = time.time()
        indices = train[i:i+batch_size]

        # Input should be of size: (batch, channels, samples)
        batch_data = get_data_index(indices)
        batch_X = batch_data['x']
        batch_y = batch_data['y']
#         print(batch_X.shape, batch_y.shape)

        # Send training data to CUDA
        if device is not "cpu":
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if np.isnan(running_loss):
            break
        running_time += time.time()-tstart
        nsamp += len(indices)
        if (time.time()-t0train)>5:
            print('Epoch %d, i=%d/%d, loss=%3.8f, t=%3.3f, %3.5f s/sample' % (epoch, i, len(train), 
                                                                              running_loss/nsamp, running_time, running_time/nsamp))
            t0train=time.time()
            print(outputs[0,:])

    loss_train = np.append(loss_train,running_loss/nsamp)
    scheduler.step(running_loss/nsamp)

    net.eval()
    running_loss_test = 0.0
    nsamp = 0
    for i in range(0, len(test), batch_size):
        indices = test[i:i+batch_size]

        # Input should be of size: (batch, channels, samples)
        batch_data = get_data(ensids[indices])
        batch_X = batch_data['x']
        batch_y = batch_data['y']

        # Send training data to CUDA
        if device is not "cpu":
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)

        outputs = net(batch_X)
        loss = criterion(outputs, batch_y)
        running_loss_test += loss.item()
        nsamp += len(indices)
    loss_test = np.append(loss_test,running_loss_test/nsamp)

    if (time.time()-t0)>5:
        plt.clf()
        plt.semilogy(loss_train,'o-',label='Train')
        plt.semilogy(loss_test,'o-',label='Test')
        plt.legend()
        display.clear_output(wait=True)
        display.display(plt.gcf())
        print('**** Epoch %d, LR=%3.3f, loss_train=%3.8f, loss_test=%3.8f, time = %3.5f s/epoch' % (epoch, 
                                                                                                    optimizer.state_dict()['param_groups'][0]['lr'], 
                                                                                                    loss_train[-1], 
                                                                                                    loss_test[-1], 
                                                                                                    (time.time()-t0)/epoch))
        print(outputs[0,:])
        t0=time.time()

#     torch.save({
#             'epoch': epoch,
#             'model_state_dict': net.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss_train': loss_train,
#             'loss_test': loss_test,
#             }, fn_save)
#     print('Saved data: %s' % fn_save)

In [ ]:
net.eval()
indices = test

yyhat = {'y':[], 'yhat':[]}
for ensid in ensids[indices]:
    # Input should be of size: (batch, channels, samples)
    batch_data = get_data([ensid])
    batch_X = batch_data['x']
    batch_y = batch_data['y']

    # Send training data to CUDA
    if device is not "cpu":
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

    outputs = net(batch_X)
    yyhat['yhat'].append(outputs.detach().cpu().numpy())
    yyhat['y'].append(batch_y)
yyhat['yhat'] = np.stack(yyhat['yhat']).squeeze()

In [ ]:
plt.plot(yyhat['yhat'].T,'.');

In [6]:
ensids[indices]

NameError: name 'ensids' is not defined